# Lesson 8: Biophysics of Neurons

Let's review what we've covered so far:

We've worked a lot with **rate models**, which are based on a fairly abstract concept of neurons as integrators. The power of this approach is in its simplicity. Because rate models are (mostly) linear, we can intuitively understand what they are doing, and we can usually fit the models to real data using standard, well-understood statistical methods.

We then spent some time with **decoding models**, which are also rather abstract in that they only seek to understand the probabilistic relationship between stimulus and response, without necessarily invoking an intervening model of how spikes are generated.

Now, we are going to dive into a more realistic view of how neurons work. As you might expect, this increased realism comes at the cost of more complexity. Sometimes it is worth it, and sometimes it is not.

We'll begin by discussing the electrical dynamics of neurons, and then work our way into the mathematics of dynamical systems.

### Before class

Read chapters 2-3 of *Principles of Computational Modelling in Neuroscience* (available as an e-book through the library or on Collab). Most of the figures in this notebook come from this text.


In [ ]:
# load matplotlib inline mode
%matplotlib inline

# import some useful libraries
import numpy as np                # numerical analysis linear algebra
import matplotlib as mpl
import matplotlib.pyplot as plt   # plotting

# set some style options
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.aspect'] = 'auto'
mpl.rcParams['image.cmap'] = 'jet'

## Reversal potential

- Cell membranes are **semipermeable**:
    - Ion channels are **selective** for certain ions
    - Negatively-charged, large macromolecules like proteins and nucleic acids are unable to exit the cell
- Semipermeability results in an **electrochemical gradient**
    - Ions flow across the membrane until drift and diffusion balance out
    
<img src="images/l10_semipermeable.png" alt="semipermeable membrane" style="width: 400px;"/>

- The voltage at which net current/flux is zero is the **reversal potential**. Follow along on the board as we derive a solution to this value.

## Nernst equation

The reversal potential for any charged ion, $E_\mathrm{X}$, is given by the following equation:

$$
E_\mathrm{x} = \frac{RT}{z_\mathrm{X}F} \log \frac{[\mathrm{X}]_\mathrm{out}}{[\mathrm{X}]_\mathrm{in}}
$$

### Exercise 1

Given the following equilibrium concentrations, complete the code cell below to print out the reversal potential for each of the ions.

|Ion|$K^+$|$Na^+$|$Cl^-$|$Ca^{2+}$|
|---|-----|------|------|---------|
|concentration inside (mM)|400|50|40|0.0001|
|concentration outside (mM)|20|440|560|10|

In [ ]:
from scipy import constants
F = constants.physical_constants["Faraday constant"][0] # 
R = constants.physical_constants["molar gas constant"][0]
T = 25 + 273.15 # Kelvin

## Current and voltage

Once we know the reversal potential for an ion, how do we figure out how much of it is flowing across the membrane?

First, for a given ion, what matters is the difference between the potential of the membrane ($V$) and the ion's reversal potential ($E_\mathrm{X}$). If $E_\mathrm{X} - V = 0$, then the ion is in equilibrium and there is no net current.

Second, we need to know the ability of the ion channels in the membrane to conduct ions. 

In an ideal world, this conductance ($g$, measured in *siemens* [S]) would be constant across all voltages, and the current would be a linear function of the voltage difference:

$$
I_\mathrm{X} = g_\mathrm{X} (E_\mathrm{X} - V)
$$

Resistance ($R$), which is more familiar to electrical engineers, is simply the reciprocal of conductance. Resistance is measured in ohms ($\Omega$).

### Exercise 2

Using the reversal potential for potassium you calculated above, and a conductance of 1.3 nS, generate a plot of current as a function of voltage from -120 mV to 40 mV (this kind of graph is called an **I-V plot**). Show current in units of pA.

## Resting potential

If a membrane is permeable to only one kind of ion, then the equilibrium or **resting potential** is simply the reversal potential for that ion.

However, most cells are permeable to more than one kind of ion, so how do we determine the resting potential in this case?

As before, it will be the voltage at which **net** total current is zero. However, the flux of different ions will not.

<img src="images/l10_resting_flux.png" alt="resting flux" style="width: 400px;"/>

**Free response**: Is the resting potential an equilibrium? Why or why not? If not, how is the cell able to maintain a resting state? Enter your answer in the cell below.

### Resting potential: Thevenin approximation

We will avoid the complexity of GHK for now and instead use an approximation based on ohmic currents in an equivalent electrical circuit.

<img src="images/l10_membrane_circuit.png" alt="equivalent electrical circuit" style="width: 400px;"/>

Here, we can see that there are four currents in parallel: the capacitative current of the membrane (we'll deal with this later), the sodium current, the potassium current, and any external current we might inject through an electrode.

According to [Thevenin's theorem](https://en.wikipedia.org/wiki/Th%C3%A9venin%27s_theorem), any circuit of resistors and voltage sources can be simplified into a single resistor and voltage source. That means we can combine the sodium and potassium currents into a single **leak current**, with the following properties:

$$
E_\mathrm{m} = \frac{g_\mathrm{Na}E_\mathrm{Na} + g_\mathrm{K}E_\mathrm{K} + \cdots}{g_\mathrm{K} + g_\mathrm{Na} + \cdots}
$$

$$
1/R_\mathrm{m} = g_\mathrm{m} = g_\mathrm{K} + g_\mathrm{Na} + \cdots
$$

### Exercise 3

You patch a neuron and empirically determine that its resting potential ($E_\mathrm{m}$) is -60 mV and its membrane resistance ($R_\mathrm{m}$) is 200 MΩ (megaohms). Using the reversal potential values you calculated earlier and the formulas above, determine $g_\mathrm{Na}$ and $g_\mathrm{K}$. This is a fairly simple linear algebra problem. Show your work or code below. 


## Putting it together

Let's return to the equivalent circuit for the membrane.

<img src="images/l10_membrane_circuit.png" alt="equivalent electrical circuit" style="width: 400px;"/>

The **law of current conservation** says that the total current in all branches of the circuit has to sum to zero.

$$
I_\mathrm{C} + I_\mathrm{K} + I_\mathrm{Na} + I_\mathrm{e} = 0
$$

$I_\mathrm{e}$ is the current through the electrode (which we control) and $I_\mathrm{C}$ is the current through the capacitor. But how can current flow through a capacitor?

It doesn't really flow *through* the capacitor, but there can be a current if the charge on the membrane is changing. Recall that $q = CV$, and that current is the change in charge over time. If we take the derivative of this equation with respect to time, we get

$$
I_\mathrm{C} = \frac{dq}{dt} = C \frac{dV}{dt}
$$

Rearranging the first equation and using our approximation for the ion channel currents then gives us:

$$
C \frac{dV}{dt} = g_\mathrm{m} (E_\mathrm{m} - V) + I_\mathrm{e}(t)
$$

### Differential equations

$$
C \frac{dV}{dt} = g_\mathrm{m} (E_\mathrm{m} - V) + I_\mathrm{e}
$$

This formula is a **differential equation**: it describes how $V$ changes in time. Because $V$ only changes with time, and because $dV/dt$ is a linear function of $V$, we can further describe this as an **ordinary first-order differential equation**, or ODE.

It happens that this ODE has an exact (or **analytical**) solution that we can obtain through separation of variables:

$$
V(t) = E_\mathrm{m} + \frac{I_\mathrm{e}}{g_\mathrm{m}} \left(1 - \exp \frac{-t}{C/g_\mathrm{m}}\right)
$$

Let's look at how this model responds to a constant injected current:

In [ ]:
dt = 0.1 # ms; time step of the simulation
T  = 200 # ms; duration of the simulation
gm = 1.3 # nS
Cm = 20.0 # pF
Em = -62 # mV
t = np.arange(0, T, dt) # ms
Ie = 30 # pA
V = Em + Ie / gm * (1 - np.exp(-t / Cm / gm))

plt.plot(t, V)
plt.xlabel("Time (ms)")
plt.ylabel("V_m (mV)")

Note the unit conversions. Once we move into the realm of biophysical realism, we have to be more careful.

- pA / nS = mV
- pF / nS = ms

What does this graph show us? 

- When a constant current is injected into the cell, the membrane voltage shifts to a new steady-state value, which is $E_\mathrm{m} + I_\mathrm{e} / g_\mathrm{m}$, the point at which current flow out of the ion channels balances current entering from the electrode.
- The membrane voltage dynamics show an **exponential decay** to the new steady-state value, with a time-constant of $\tau = C_\mathrm{m} / g_\mathrm{m} = C_\mathrm{m} R_\mathrm{m}$.

### Numerical integration

We'll explore the properties of our model more later, but first, we're going to learn how to **numerically** integrate the ODE. 

This is almost always necessary because most ODEs for complex systems do not have analytical solutions.

There are many algorithms for numerical integration, but they all involve the same basic principles:

- start from initial conditions $V(t = 0)$
- calculate derivative: $V'(0) = dV/dt$
- for a small time step $\Delta$, $V(\Delta) \approx V(0) + V'(0) \Delta$
- repeat for $t = \Delta, 2\Delta, \ldots$

<img src="images/l10_euler_method.png" alt="euler_method" style="width: 200px;"/>

As the graph shows, numerical integration approximates the true integral by a series of short lines.

Shorter step sizes ($\Delta$) generally lead to smaller errors.

### Numerical integration in Python

Unless you're just trying to learn how numerical integration works, you should always use a library. The scipy package has an excellent set of [integration functions](https://docs.scipy.org/doc/scipy/reference/integrate.html).

Almost every solver will require you to provide a function that evaluates the system of ODEs at a given point, plus initial conditions. Here's an example for our simple membrane model:


In [ ]:
from scipy import integrate
# We're going to generate an array with values of I at each time point
I = np.zeros_like(t)
I[10:1000] = Ie 

# this is the derivative
def dVdt(t, y):
    # look up the value of the injected current at time t
    if t < 0 or t >= T:
        It = 0
    else:
        idx = int(t / dt)
        It = I[idx]
    return (gm * (Em - y[0]) + It) / Cm

ret = integrate.solve_ivp(dVdt, (0, T), [Em], method='RK23', t_eval=t)
plt.plot(ret.t, ret.y.squeeze())

### Exercise 4

Let's see what happens when a neuron is stimulated with something other than a step current.

1. Generate 10 s of white noise with mean 0 and standard deviation 10 pA.
2. Use numerical integration to determine V(t) over this interval and plot it.

In [ ]:
T = 10000
t = np.arange(0, T, dt)
# generate your stimulus here and assign it to I

ret = integrate.solve_ivp(dVdt, (0, T), [Em], method='RK23', t_eval=t)
V = ret.y.squeeze()
plt.plot(ret.t, V)

3. Calculate and plot the correlation between V(t) and I(t). What does the filter look like?

## The Hodgkin-Huxley model

We can now write the complete model as a set of four ODEs:

\begin{align}
C_\mathrm{m} \dot{V} & = g_L(E_L - V) + \overline{g}_\mathrm{Na}m^3h(E_\mathrm{Na} - V) + \overline{g}_\mathrm{K}n^4(E_\mathrm{K} - V) + I_e\\
\dot{m} & = \frac{m_\infty(V) - m}{\tau_m(V)} \\
\dot{h} & = \frac{h_\infty(V) - h}{\tau_h(V)} \\
\dot{n} & = \frac{n_\infty(V) - n}{\tau_n(V)} \\
\end{align}

For simplicity, I'm leaving out the $(t)$ for the four state variables $V$, $m$, $h$, and $n$, and I'm using a dot to denote the time derivative.

Note that some of the activation variables are raised to a power. Hodgkin and Huxley found these powers empirically by doing some heroic curve-fitting.

There are many parameters in this model: $C_\mathrm{m}, E_L, g_L, E_\mathrm{Na}, \overline{g}_\mathrm{Na}, E_\mathrm{K}, \overline{g}_\mathrm{K}$, plus all the parameters that go into describing the shapes of the kinetic functions. These parameters usually have to found empirically as well.

### Simulating the model

In principle, integrating the HH model is the same as integrating our simple membrane model. We just have a vector of four numbers as our current state, and a lot more parameters. 

It's not terrible to implement, just a lot of bookkeeping.


In [ ]:
C_m  =   1.0 # membrane capacitance, in uF/cm^2
g_Na = 120.0 # Sodium (Na) maximum conductance, in mS/cm^2
g_K  =  40.0 # Potassium (K) maximum conductance, in mS/cm^2
g_L  =   0.3 # Leak maximum conductance, in mS/cm^2
E_Na =  50.0 # Sodium (Na) Nernst reversal potential, in mV
E_K  = -77.0 # Potassium (K) Nernst reversal potential, in mV
E_L  = -54.4 # Leak Nernst reversal potential, in mV

def alpha_m(V):
    return 0.1*(V+40.0)/(1.0 - np.exp(-(V+40.0) / 10.0))

def beta_m(V):
    return 4.0*np.exp(-(V+65.0) / 18.0)

def alpha_h(V):
    return 0.07*np.exp(-(V+65.0) / 20.0)

def beta_h(V):
    return 1.0/(1.0 + np.exp(-(V+35.0) / 10.0))

def alpha_n(V):
    return 0.01*(V+55.0)/(1.0 - np.exp(-(V+55.0) / 10.0))

def beta_n(V):
    return 0.125*np.exp(-(V+65) / 80.0)

dt = 0.1
T = 500.
t = np.arange(0, T, dt)
I = np.zeros_like(t)

def deriv(t, X):
    """The derivative function"""
    if t < 0 or t >= T:
        It = 0
    else:
        idx = int(t / dt)
        It = I[idx]
    # unpack the state vector
    V, m, h, n = X
    dVdt = 1. / C_m * (It + g_L * (E_L - V) + g_Na * m**3 * h * (E_Na - V) + g_K * n**4 * (E_K - V))
    dmdt = alpha_m(V)*(1.0-m) - beta_m(V)*m
    dhdt = alpha_h(V)*(1.0-h) - beta_h(V)*h
    dndt = alpha_n(V)*(1.0-n) - beta_n(V)*n
    return (dVdt, dmdt, dhdt, dndt)

In [ ]:
X0 = [-65, 0.05, 0.6, 0.32]
I[1000:3000] = 50.
ret = integrate.solve_ivp(deriv, (0, T), X0, method='LSODA', t_eval=t)
plt.plot(ret.t, ret.y[0])

### Inspecting the model

If everything worked correctly, you should see a train of spikes in the plot.

We can do more, though: we can inspect the underlying variables in the model in a way that we could never do with a real neuron.

Let's look at how $m$, $h$, and $n$ change during a spike, and how this relates to the voltage-gated $I_\mathrm{Na}$ and $I_\mathrm{K}$ currents.

In [ ]:
V, m, h, n = ret.y
I_Na = g_Na * m **3 * h * (E_Na - V)
I_K = g_K * n**4 * (E_K - V)
fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(9, 6))
axes[0].plot(ret.t, ret.y[0])
axes[1].plot(ret.t, np.column_stack([m, h, n]))
axes[1].legend(('m', 'h', 'n'))
axes[2].plot(ret.t, np.column_stack([I_Na, I_K]))
axes[2].legend(('I_Na', 'I_K'))
axes[1].set_xlim(99, 160);

### Exercise 5

Generate an f-I plot for this model neuron by changing the magnitude of the injected current from 0 to 20 μA/cm² in steps of 0.2 μA/cm² and measuring the spike rate for each stimulus.

I'll get you started with some code to count the number of spikes, but you need to write the loop to update the injected current magnitude and run the integration.

In [ ]:
import quickspikes as qs

def count_spikes(V):
    """Return the number of spikes in voltage trace V that exceed -10 mV"""
    det = qs.detector(-10, 40)
    spike_t = det.send(V)
    return len(spike_t)

I_values = np.arange(0, 20, 0.2)
f_values = np.zeros_like(I_values)
for i, Ie in enumerate(I_values):
    # update I with new Ie value
    # run integration
    # count spikes
    # assign to f_values[i]
    
# plot f_values vs I_values